# Run multiple rainfall events
- Explore rainfall boundary conditions using MIKE IO.
- Create a boundary condition for each event; set all inactive by default.
- Run the model for each event by activating the relevant boundary condition and updating the scenario name.

In [1]:
from pathlib import Path

rainfall_timeseries = Path("../tests/testdata/Db/Sirius/Sirius_IDF_rainfall.dfs0")
db_path = Path("../tests/testdata//Db/Sirius/Sirius.sqlite")

#### Explore rainfall boundary dfs0 file

In [2]:
import mikeio

ds = mikeio.read(rainfall_timeseries)
ds

<mikeio.Dataset>
dims: (time:22)
time: 2019-01-01 00:00:00 - 2019-01-02 00:00:00 (22 non-equidistant records)
geometry: GeometryUndefined()
items:
  0:   F=20 <Rainfall Intensity> (mm per hour) - 3
  1:   F=10 <Rainfall Intensity> (mm per hour) - 3
  2:   F=5 <Rainfall Intensity> (mm per hour) - 3
  3:   F=2 <Rainfall Intensity> (mm per hour) - 3
  4:   F=1 <Rainfall Intensity> (mm per hour) - 3
  5:   F=0.5 <Rainfall Intensity> (mm per hour) - 3
  6:   F=0.2 <Rainfall Intensity> (mm per hour) - 3
  7:   F=0.1 <Rainfall Intensity> (mm per hour) - 3
  8:   F=0.05 <Rainfall Intensity> (mm per hour) - 3

In [3]:
# File path, item name, and data type.

event_info = []
for item in ds.items:
    event_info.append(
        {
            "file_path": str(rainfall_timeseries.relative_to(db_path.parent)),
            "item_name": item.name,
            "data_type": repr(item.type),
        }
    )

event_info

[{'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=20',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=10',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=5',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=2',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=1',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.5',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.2',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.1',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.05',
  'data_type': 'Rainfall Intensity'}]

#### Create boundary conditions for each rainfall event

In [4]:
# Avoid conflict with MIKE IO - run in separate notebook
import sys
del sys.modules["mikeio"]

In [5]:
# Open MIKE+ database with MIKE+Py
import mikeplus as mp

db = mp.open(db_path)
db

Database<'Sirius.sqlite'>

In [6]:
# Create a boundary condition for each event
msm_BBoundary = db.tables.msm_BBoundary
for event in event_info:
    msm_BBoundary.insert({
        msm_BBoundary.columns.MUID              : "Rainfall " + event["item_name"],
        msm_BBoundary.columns.VariationNo       : 3, # set temporal variation to timeseries
        msm_BBoundary.columns.FlowTypeNo        : 5, # set boundary condition type to Rainfall
        msm_BBoundary.columns.LoadTypeNo        : 1, # set load type to stormwater runoff
        msm_BBoundary.columns.TSConnection      : event["file_path"],
        msm_BBoundary.columns.TimeseriesName    : event["item_name"],
        msm_BBoundary.columns.DataTypeName      : event["data_type"],
        msm_BBoundary.columns.ApplyBoundaryNo   : 0,
    })

In [7]:
# Verify
msm_BBoundary.select([
    msm_BBoundary.columns.VariationNo,
    msm_BBoundary.columns.FlowTypeNo,
    msm_BBoundary.columns.LoadTypeNo,
    msm_BBoundary.columns.TSConnection,
    msm_BBoundary.columns.TimeseriesName,
    msm_BBoundary.columns.DataTypeName,
    msm_BBoundary.columns.ApplyBoundaryNo,
]).to_pandas()

,VariationNo,FlowTypeNo,LoadTypeNo,TSConnection,TimeseriesName,DataTypeName,ApplyBoundaryNo
Rainfall F=20,3,5,1,C:\course_models\RYWM\module4_model\mikepluspy...,F=20,Rainfall Intensity,0
Rainfall F=10,3,5,1,C:\course_models\RYWM\module4_model\mikepluspy...,F=10,Rainfall Intensity,0
Rainfall F=5,3,5,1,C:\course_models\RYWM\module4_model\mikepluspy...,F=5,Rainfall Intensity,0
Rainfall F=2,3,5,1,C:\course_models\RYWM\module4_model\mikepluspy...,F=2,Rainfall Intensity,0
Rainfall F=1,3,5,1,C:\course_models\RYWM\module4_model\mikepluspy...,F=1,Rainfall Intensity,0
Rainfall F=0.5,3,5,1,C:\course_models\RYWM\module4_model\mikepluspy...,F=0.5,Rainfall Intensity,0
Rainfall F=0.2,3,5,1,C:\course_models\RYWM\module4_model\mikepluspy...,F=0.2,Rainfall Intensity,0
Rainfall F=0.1,3,5,1,C:\course_models\RYWM\module4_model\mikepluspy...,F=0.1,Rainfall Intensity,0
Rainfall F=0.05,3,5,1,C:\course_models\RYWM\module4_model\mikepluspy...,F=0.05,Rainfall Intensity,0


#### Run each event

In [8]:
# Only one simulation exists.
msm_Project = db.tables.msm_Project
msm_Project.select().to_pandas()

,MUID,Sqn,IncludeToBatchNo,ScenarioName,ActiveProject,Enable_Catchment,Enable_CS,Enable_River,Enable_2DOverland,Enable_RR,...,DecouplingPath,ELIntegrationNo,ELUpdateFrequency,M2DADDtMax,M2DADDtMin,M2DADMaxCFL,MJLFileName,HDOutputNo,HDFolderPath,SummaryID
Sirius_1_DEMO,Sirius_1_DEMO,1,1,Base,1,0,1,0,0,0,...,None,3,1,10.0,0.01,0.8,None,1,None,Summary_Sirius_1


In [9]:
msm_BBoundary.select([msm_BBoundary.columns.ApplyBoundaryNo]).to_pandas()

,ApplyBoundaryNo
Rainfall F=20,0
Rainfall F=10,0
Rainfall F=5,0
Rainfall F=2,0
Rainfall F=1,0
Rainfall F=0.5,0
Rainfall F=0.2,0
Rainfall F=0.1,0
Rainfall F=0.05,0


In [10]:
boundaries = msm_BBoundary.get_muids()
boundaries

['Rainfall  F=20',
 'Rainfall  F=10',
 'Rainfall  F=5',
 'Rainfall  F=2',
 'Rainfall  F=1',
 'Rainfall  F=0.5',
 'Rainfall  F=0.2',
 'Rainfall  F=0.1',
 'Rainfall  F=0.05']

In [11]:
# Run the simulation for each boundary
boundary_result_files = {}
for boundary in boundaries:
    # apply boundary
    msm_BBoundary.update({"ApplyBoundaryNo": 1}).where("MUID = :muid", {"muid": boundary}).execute()

    # update simulation name
    simulation = msm_Project.get_muids()[0]
    msm_Project.update({"MUID": boundary}).where("MUID = :muid", {"muid": simulation}).execute()

    # run simulation
    result_files = db.run(simulation_muid=boundary)
    boundary_result_files[boundary] = result_files

    # unapply boundary
    msm_BBoundary.update({"ApplyBoundaryNo": 0}).where("MUID = :muid", {"muid": boundary}).execute()

In [12]:
# Print result files relative to db path
for boundary, result_files in boundary_result_files.items():
    result_files = [r.relative_to(db.db_path.resolve().parent) for r in result_files]
    result_files = [r.as_posix() for r in result_files]
    print(f"Boundary: {boundary}")
    print(f"Result files: {result_files}")
    print()

Boundary: Rainfall  F=20
Result files: ['Sirius_m1d - Result Files/Rainfall  F=20BaseDefault_Network_HD.res1d']

Boundary: Rainfall  F=10
Result files: ['Sirius_m1d - Result Files/Rainfall  F=10BaseDefault_Network_HD.res1d']

Boundary: Rainfall  F=5
Result files: ['Sirius_m1d - Result Files/Rainfall  F=5BaseDefault_Network_HD.res1d']

Boundary: Rainfall  F=2
Result files: ['Sirius_m1d - Result Files/Rainfall  F=2BaseDefault_Network_HD.res1d']

Boundary: Rainfall  F=1
Result files: ['Sirius_m1d - Result Files/Rainfall  F=1BaseDefault_Network_HD.res1d']

Boundary: Rainfall  F=0.5
Result files: ['Sirius_m1d - Result Files/Rainfall  F=0.5BaseDefault_Network_HD.res1d']

Boundary: Rainfall  F=0.2
Result files: ['Sirius_m1d - Result Files/Rainfall  F=0.2BaseDefault_Network_HD.res1d']

Boundary: Rainfall  F=0.1
Result files: ['Sirius_m1d - Result Files/Rainfall  F=0.1BaseDefault_Network_HD.res1d']

Boundary: Rainfall  F=0.05
Result files: ['Sirius_m1d - Result Files/Rainfall  F=0.05BaseDefault

In [13]:
db.close()